# Download All Feature Layers from ArcGIS Online

This notebook demonstrates how to:
1. Import Libraries and Sign in to ArcGIS Online using your credentials.
2. Query all feature layers available in your organization.
3. Save Feature Layer information to a local csv.
4. Export and Download FGDBs to a local folder.
5. Delete exported ArcGIS Online FGDB.

The notebook uses the `arcgis` Python API to interact with ArcGIS Online.

## Step 1: Import Required Libraries & Sign in to ArcGIS Online


Use your ArcGIS Online credentials to sign in. The `getpass` library is used to securely input your password. The `GIS` object from the `arcgis` library is used to establish the connection.

In [ ]:
import os
import getpass
import pandas as pd
from arcgis.gis import GIS
from datetime import datetime

In [ ]:

# Sign in to ArcGIS Online
username = input("Enter your ArcGIS Online username: ")
password = getpass.getpass("Enter your ArcGIS Online password: ")
gis = GIS("https://www.arcgis.com", username, password, verify_cert=False)


## Step 2: Query Feature Layers

Search for all feature layers in your ArcGIS Online organization. The `search` method of the `GIS` object is used to retrieve items of type `Feature Layer`. Metadata such as the layer ID, name, description, owner, creation date, modification date, and URL is extracted and stored in a list.

In [ ]:
fl_content = gis.content.search('Feature Layer', max_items=9999)

print(f"Feature Layer count: {len(fl_content)}")

## Step 3: Create and Save DataFrame
The metadata of the queried feature layers is organized into a pandas DataFrame for easy viewing and manipulation. The DataFrame includes the following columns:
- `id`: The unique ID of the feature layer.
- `name`: The name of the feature layer.
- `description`: A brief description of the feature layer.
- `owner`: The owner of the feature layer.
- `created`: The creation date of the feature layer (in DD/MM/YYYY format).
- `modified`: The last modification date of the feature layer (in DD/MM/YYYY format).
- `url`: The URL of the feature layer.

The DataFrame is then saved to a CSV file named `feature_layers_metadata.csv` for future reference.


In [ ]:

# Query all feature layers in the organization
print("Querying feature layers...")
feature_layers = []
for item in fl_content:
    if "Feature" in item.type:
        feature_layers.append({
            "id": item.id,
            "name": item.title,
            "description": item.description,
            "owner": item.owner,
            "created": datetime.utcfromtimestamp(item.created / 1000).strftime('%d/%m/%Y'),
            "modified": datetime.utcfromtimestamp(item.modified / 1000).strftime('%d/%m/%Y'),
            "url": item.url
        })

# Create a DataFrame with metadata
df = pd.DataFrame(feature_layers)
df.head()


In [ ]:
# Save the metadata to a CSV file
print("Saving metadata to CSV...")
df.to_csv("feature_layers_metadata.csv", index=False)
print("Metadata saved to feature_layers_metadata.csv")

## Step 4: Export and Download Feature Layers

A local folder named `downloads` is created in the current working directory. This folder will be used to store the downloaded FGDBs.

Each feature layer is exported as a File Geodatabase (FGDB) using the `export` method of the `Item` object. The exported FGDB is then downloaded to the `downloads` folder. After the download is complete, the exported item is deleted to clean up resources in ArcGIS Online.

### Error Handling
If an error occurs during the export or download process, it is caught and logged, allowing the script to continue processing the remaining feature layers.

In [ ]:
# Create a local download folder in the current notebook directory
download_folder = os.path.join(os.getcwd(), "downloads")
os.makedirs(download_folder, exist_ok=True)

# Export and download each feature layer as an FGDB
print("Exporting and downloading feature layers...")
for index, row in df.head().iterrows(): 
    try:
        item = gis.content.get(row["id"])
        print(f"Exporting {row['name']}...")
        export_item = item.export(title=f"{row['name']}_FGDB", export_format="File Geodatabase")
        print(f"Downloading {row['name']}...")
        export_item.download(save_path=download_folder)
        export_item.delete()  # Clean up the export item after download
        print(f"Downloaded {row['name']} to {download_folder}")
    except Exception as e:
        print(f"Failed to export or download {row['name']}: {e}")

print("All feature layers processed.")


## Conclusion

This notebook provides a complete workflow for downloading all feature layers from ArcGIS Online as FGDBs. The downloaded FGDBs are stored in a local folder for further use. The process is automated and handles errors gracefully, ensuring that all accessible feature layers are processed.